In [1]:
import cv2
import numpy as np
from collections import deque

In [3]:
input_video  = r"C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Input Video\solidWhiteRight.mp4"
output_video = r"C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Control Simulation\Output.mp4"

In [5]:
#Load the Object Detection Model (Part 2)
model = r"C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Basic Perception\Models\frozen_inference_graph.pb"
config = r"C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Basic Perception\Models\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"
labels = r"C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Basic Perception\Models\coco.names"

with open(labels, "r") as f:
    classNames = f.read().rstrip("\n").split("\n")

net = cv2.dnn_DetectionModel(model, config)
net.setInputSize(320, 320)
net.setInputScale(1/127.5)
net.setInputMean((127.5,127.5,127.5))
net.setInputSwapRB(True)

< cv2.dnn.Model 0000027CD6DBD070>

In [7]:
#PART-2 — Object Detection Functions
#Detection Function
def detect_objects(image, conf=0.5):
    classIds, confs, boxes = net.detect(image, confThreshold=conf)
    objs = []

    if len(classIds) != 0:
        for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), boxes):
            label = classNames[classId - 1]

            if label not in ["car", "truck", "bus"]:
                continue

            objs.append((label, float(confidence), box))
    return objs

In [9]:
#Draw Function
def draw_objects(image, objects):
    for label, confidence, box in objects:
        x, y, w, h = map(int, box)
        cv2.rectangle(image, (x,y),(x+w,y+h),(0,255,0),2)
        cv2.putText(image, f"{label} {confidence:.2f}", (x,y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,255,0),2)
    return image

In [11]:
#PART-1 — Lane Detection Functions
def canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 50, 150)
    return edges

def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([[(0,height), (image.shape[1],height), (image.shape[1]//2, height//2)]])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    return cv2.bitwise_and(image, mask)

def display_lines(image, lines):
    line_img = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2 = line.reshape(4)
            cv2.line(line_img,(x1,y1),(x2,y2),(255,0,0), 8)
    return line_img

def average_slope_intercept(image, lines):
    if lines is None:
        return None

    # FIX: flatten shape (N,1,4) → (N,4)
    lines = lines.reshape(-1, 4)

    left_fit = []
    right_fit = []

    for x1, y1, x2, y2 in lines:
        if x2 == x1:
            continue
        slope = (y2 - y1) / (x2 - x1)
        intercept = y1 - slope * x1

        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))

    line_points = []

    for group in (left_fit, right_fit):
        if len(group) > 0:
            slope, intercept = np.mean(group, axis=0)
            y1 = image.shape[0]
            y2 = int(y1 * 0.6)

            x1 = int((y1 - intercept) / slope)
            x2 = int((y2 - intercept) / slope)

            line_points.append([x1, y1, x2, y2])

    return np.array(line_points)


def lane_pipeline(image):
    edges = canny(image)
    cropped = region_of_interest(edges)
    lines = cv2.HoughLinesP(cropped, 1, np.pi/180, 100,
                            minLineLength=40, maxLineGap=50)
    averaged = average_slope_intercept(image, lines)
    line_img = display_lines(image, averaged)
    combo = cv2.addWeighted(image, 0.8, line_img, 1, 1)
    return combo

In [13]:
#PART-3 — Control Functions
def compute_steering(lane_center_x, frame_width):
    car_center_x = frame_width // 2
    offset = lane_center_x - car_center_x
    steering_angle = (offset / frame_width) * 60

    if steering_angle > 10:
        decision = "Turn Right"
    elif steering_angle < -10:
        decision = "Turn Left"
        decision = "Turn Left"
    else:
        decision = "Go Straight"

    return steering_angle, decision

In [15]:
def compute_speed(objects):
    speed = 30
    decision = "Normal Speed"
    
    for label, conf, box in objects:
        x, y, w, h = box
        if w*h > 30000:    # Means obstacle very close
            speed = 0
            decision = "BRAKE!"
            break
    
    return speed, decision

In [17]:
def control_decision(lane_center_x, frame_width, objects):
    steering, steering_decision = compute_steering(lane_center_x, frame_width)
    speed, speed_decision = compute_speed(objects)
    
    final = f"{steering_decision} | {speed_decision}"
    return steering, speed, final

In [23]:
#Overlay Text on Frame
def draw_control_info(frame, steering, speed, decision):
    text = f"Steering: {steering:.1f} | Speed: {speed} | Action: {decision}"
    cv2.putText(frame, text, (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)
    return frame

In [25]:
#Final Video Pipeline (All Parts Combined)

cap = cv2.VideoCapture(input_video)

w = int(cap.get(3))
h = int(cap.get(4))
fps = int(cap.get(5)) or 30

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video, fourcc, fps, (w, h))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # ---------- PART 1 (Lane Detection) ----------
    lane_result = lane_pipeline(frame)
    lane_center_x = w // 2   
    
    # ---------- PART 2 (Object Detection) ----------
    objects = detect_objects(frame)
    frame = draw_objects(frame, objects)

    # ---------- PART 3 (Control Simulation) ----------
    steering, speed, final_decision = control_decision(
        lane_center_x, w, objects
    )

    frame = draw_control_info(frame, steering, speed, final_decision)

    out.write(frame)

cap.release()
out.release()
print("Completed!")
print("Video saved:", output_video)

Completed!
Video saved: C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Control Simulation\Output.mp4
